# Streams and Roofs

In this week's assignment we are going to make some roofline diagrams for some $n$-body problems.

This week's assignment is meant to be run on a node with a Tesla P100 GPU.

A reminder: when you are running a job to complete this week's assignment, you should make sure that you have requested exclusive access to a node, and that you have requested access to all CPU cores of this node.

**Due: Thursday, September 12, before class**

Let's load in our class module:

In [1]:
module use $CSE6230_DIR/modulefiles

In [2]:
module load cse6230

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |


In [3]:
module list

Currently Loaded Modulefiles:
  1) curl/7.42.1
  2) git/2.13.4
  3) python/3.6
  4) /nv/coc-ice/tisaac3/opt/pace-ice/modulefiles/jupyter/1.0
  5) intel/16.0
  6) cuda/8.0.44
  7) /nv/coc-ice/tisaac3/opt/pace-ice/modulefiles/hpctoolkit/2018.18
  8) impi/5.1.1.109
  9) cse6230/core(default)


And verify that we're running where we expect to run:

In [ ]:
nvidia-smi

Great!

Now, about the $n$-body simulations we're going to run: a classical $n$-body simulation has each body, or *particle*, interacting with each other, for $n(n+1)/2$ total interactions.  That hardly matches up to the streaming kernels we've been talking about!  So we're going to simplify a bit.

We are going to simulate $n$ infinitesimal particles circling around an infinitely massive sun at the origin.  In this system, the sun is unmoved, and the particles are not affected by each other.

We're going to normalize our coefficients and say that each particle is an ordinary differential equation with *six* components: three of position $X=(x, y, z)$ and three of velocity $U=(u, v, w)$.  The position, is changed by the velocity, of course, but the velocity changes under acceleration that depends on position:

$$\begin{aligned} \dot{X} &= V \\ \dot{V} &= - \frac{X}{|X|^3}.\end{aligned}$$

To discretize this differential equation, we are going to use a time stepping method called the Verlet leap-frog method, which is good for calculating long simulations of stable orbits.  Given a time step length `dt`, our pseudocode for one time step for one particle looks like the following:

1. `X += 0.5 * dt * V`
2. `R2 = X . X` (dot product)
3. `R = sqrt (R2)`
4. `IR3 = 1. / (R2 * R)`
5. `V -= X * dt * IR3`
6. `X += 0.5 * dt * V`

**Question 1.** Assuming `sqrt` and `div` count for one flop each, and assuming `x, y, z` and `u, v, w` are **double-precision** floating point
numbers, **estimate the arithmetic intensity of a *particle time step***.  You should ignore the time it takes to load `dt`.  Your answer should have units of flops / byte.  Give your answer in a new cell below this one, and show how you arrived at that number.

$\frac{7}{24}$ flops/byte

Suppose our compiler is smart enough to keep reused value in register

1. `X += 0.5 * dt * V`: 1 flop (`0.5 * dt`) + 3 flops (`k * V`) + 3 (`+`) = 7 flops; read 6 variables
2. `R2 = X . X` (dot product): 5 flops
3. `R = sqrt (R2)`: 1 flop
4. `IR3 = 1. / (R2 * R)`: 2 flops
5. `V -= X * dt * IR3`: 7 flops
6. `X += 0.5 * dt * V`: 6 flops; write 6

in total 28 flops

each `double` variable is 8 bytes, in total 12 times r/w, 96 bytes

**Question 2.** Using the peak theoretical **double-precision** flop/s of this node (flop/s on the CPUs and GPU combined), calculated the same way as in the last assignment, and reported peak memory bandwidths from the manufacturers, **estimate the system balance of CPUs and the GPU of this node separately**.  Note that the bandwidth estimate from intel will be for one socket (4 cores) with attached memory, and our node has two such sockets.

In [39]:
cat /proc/cpuinfo | grep "core id"
# nvidia-smi

core id		: 0
core id		: 1
core id		: 2
core id		: 3
core id		: 0
core id		: 1
core id		: 2
core id		: 3


cpu peak flop: 8 * 2.6G * 16 (double precision) = 332 Gflop/s

gpu peak flop: reported by nvidia, 4700 Gflop/s

cpu peak bandwidth: 68.3 * 2 = 136.4 GB/s 

gpu peak bandwidth: 32 Gb/s (PCIe), 732GB/s (HBM)

cpu balance point: 332/136.4 = 2.44 flop/byte

gpu balance point: 146.88 flop/byte(PCIe), 6.42 flop/byte

Last week, we didn't take the peak flop/s values from the manufacturers at face value, and this week we are not going to take the peak Gbyte/s for granted either.  Last week we used a custom benchmark in our calculations; this week we will use an industry standard: the
[STREAM benchmark](https://www.cs.virginia.edu/stream/ref.html).

We can run the stream benchmark on the CPUs for this assignment with a makefile target:

In [ ]:
OMP_NUM_THREADS=8 make runstream STREAM_N=40000 COPTFLAGS="-O3"

The `STREAM_N` argument will control the size of the stream arrays.

**Question 3:** Modify the invocation of `make runstreams` by modifying the values of
`STREAM_N`, `COPTFLAGS` (optimization flags), `OMP_NUM_THREADS` and/or `OMP_PROC_BIND` (the openMP environment variables) to get the largest streaming bandwidth from main memory that you can for this node.

[The OpenMP environment variables were not defined by me in the Makefile: they are environment variables that will be detected by the OpenMP runtime when an OpenMP program begins.  You should put them _before_ the make command, e.g. `OMP_NUM_THREADS=5 make runstream STREAM_N=40000000`]

- Follow the directions in the output of the file and make sure you are testing streaming bandwidth from memory and not from a higher level of cache.
- You should try to get close to the same bandwidth for all tests:

- There are two variables in the openMP environment you should care about, OMP_NUM_THREADS, which is self explanatory, and OMP_PROC_BIND is discussed [here](http://pages.tacc.utexas.edu/~eijkhout/pcse/html/omp-affinity.html).  **You should try to use as few threads as possible** to achieve peak bandwidth.

In [ ]:
for i in {6..8}
do
echo $i
OMP_NUM_THREADS=$i
OMP_PROC_BIND=SPREAD
# unset OMP_PROC_BIND
make clean
make runstream STREAM_N=40000000 COPTFLAGS="-O3" | grep -2 Scale:
done

In [14]:
OMP_NUM_THREADS=7
OMP_PROC_BIND=SPREAD
make clean
make runstream STREAM_N=40000000 COPTFLAGS="-O3" | grep -2 Scale:
# 4 chaanels/socket * 2 sockets giving 8 channels
# seems value around 6,7,8 is not so stable, 7 gives best value for most times

rm -f *.o cloud stream stream2 streamcu streamcu2
Function    Best Rate MB/s  Avg time     Min time     Max time
Copy:           48955.1     0.016244     0.013073     0.021038
Scale:          51969.0     0.015500     0.012315     0.018714
Add:            58249.2     0.018245     0.016481     0.019340
Triad:          55980.0     0.018290     0.017149     0.021692


**Question 4:** What does `OMP_PROC_BIND=close` mean, and why is it a bad choice, not just for this benchmark, but for any streaming kernel?

OMP_PROC_BIND: 
- `TRUE`: Enable process binding.
- `FALSE`: Disable process binding.
- `MASTER`: Bind threads to the same place as the master thread.
- `CLOSE`: Bind threads close to the master thread while still distributing threads for load balancing.
- `SPREAD`: Bind threads as evenly distributed (spread) as possible.

> On the other hand, if the two sockets have a total of sixteen cores and you define
OMP_PLACES=cores
OMP_PROC_BIND=close
then
*  thread 0 goes to core 0, which is on socket 0,
*  thread 1 goes to core 1, which is on socket 0,
*  thread 2 goes to core 2, which is on socket 0,
*  and so on, until thread 7 goes to core 7 on socket 0, and
*  thread 8 goes to core 8, which is on socket 1,
*  et cetera.

> The value  OMP_PROC_BIND=close  means that the assignment goes successively through the available places. The variable  OMP_PROC_BIND  can also be set to  spread , which spreads the threads over the places. With
OMP_PLACES=cores
OMP_PROC_BIND=spread
you find that
*  thread 0 goes to core 0, which is on socket 0,
*  thread 1 goes to core 8, which is on socket 1,
*  thread 2 goes to core 1, which is on socket 0,
*  thread 3 goes to core 9, which is on socket 1,
*  and so on, until thread 14 goes to core 7 on socket 0, and
*  thread 15 goes to core 15, which is on socket 1.

SPREAD is better because it's utilizing cpu cache stuff.

**Question 5:** I've modified the benchmark, calling it `stream2.c`.  Here's the difference, it's one line of code:

In [5]:
diff stream.c stream2.c

267d266
< #pragma omp parallel for


: 1

Copy your options for `runstream` to `runstream2` below.  The reported results should be different: why?

In [13]:
OMP_NUM_THREADS=8
OMP_PROC_BIND=SPREAD
make clean
make runstream2 STREAM_N=10000000 COPTFLAGS="-O3 -xHost" | grep -2 Scale:

rm -f *.o cloud stream stream2 streamcu streamcu2
Function    Best Rate MB/s  Avg time     Min time     Max time
Copy:           36339.9     0.006071     0.004403     0.009383
Scale:          36239.8     0.005929     0.004415     0.010027
Add:            37078.1     0.008387     0.006473     0.010596
Triad:          37371.3     0.008146     0.006422     0.009317


if we don't use omp parallel for in data initialization, then half number of processes will not have data localized, and wait for thir peers to finish, thus increase runtime roughly double

**Question 6:** Now we're going to run stream benchmarks for the GPU.  As above, modify the array size until you believe you are testing streaming bandwidth from memory and not from cache.

In [44]:
unset OMP_NUM_THREADS
for i in 7
do
echo $(bc <<< "10^$i")
make clean
make runstreamcu STREAM_N=$(bc <<< "10^$i") CUOPTFLAGS="-O3" | grep -2 Scale:
done

10000000
rm -f *.o cloud stream stream2 streamcu streamcu2
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
Function    Best Rate MB/s  Avg time     Min time     Max time
Copy:          529667.4     0.000303     0.000302     0.000306
Scale:         528000.5     0.000304     0.000303     0.000305
Add:           545600.5     0.000442     0.000440     0.000456
Triad:         545600.5     0.000441     0.000440     0.000442


**Question 7 (2 pts):** This is final time we're running a stream benchmark, I promise.  This benchmark is also for the GPU, but instead of the arrays originating in the GPUs memory, they start on the CPUs memory, and must be transfered to the GPU and back.  This mimics a common design pattern when people try to modify their code for GPUs: identify the bottleneck kernel, and try to "offload" it to the GPU, where it will have a higher throughput (once it get's there).  You don't have to modify this run, I just want you to see what bandwidths it reports:

In [10]:
make runstreamcu2 STREAM_N=1000000 | grep -2 Scale:

nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
stream2.cu(569): warning: variable "i" was set but never used

stream2.cu(569): warning: variable "i" was set but never used

Function    Best Rate MB/s  Avg time     Min time     Max time
Copy:            6123.1     0.002628     0.002613     0.002660
Scale:           6128.1     0.002617     0.002611     0.002635
Add:             6135.0     0.003929     0.003912     0.003943
Triad:           6135.0     0.003921     0.003912     0.003937


Now, with the three peak bandwidths that we have *computed* (not the reported values from question 2) -- CPU, GPU with arrays on the GPU, and GPU with arrays on the CPU -- and with the theoretical peak flop/s for the CPU and GPU, compute *effective system balances* and create a plot with rooflines for all three balances overlayed.

- The y axis should be absolute Gflop/s, not relative, so we can compare them, and should be labeled "Gflop/s"
- Label with roofline goes with which balance: "CPU", "GPU", "CPU->GPU->CPU"
- The x axis should be in units of "double precision flops / byte"

Save your plot as the jpg `threerooflines.jpg` so that it can embed in the cell below

CPU bandwith: 56GB/s
GPU bandwith: 6GB/s(PCIe), 530GB/s(HBM)

![Three rooflines](./threerooflines.png)

**Question 8 (2 pts):** Remember those particles all the way back in question 1?  Your arithmetic intensity estimate could be placed on the roofline plot for the CPUs, and you could make a judgement about whether the kernel is compute bound or memory bound.

Now let's put it to the test.  The `make runcloud` target simulates `NPOINT` particles orbiting the sun for `NT` time steps.  Because these particles are independent, you can optionally "chunk" multiple time steps for each particle independent of the other particles.  Doing this reduces the number of memory accesses per flop:  each particle stays in register for `NCHUNK` time steps.

Do your best to optimize the throughput of the simulation both in the limit of few particles and many time steps, and in the limit of many particles and few time steps.
Do that by modifying the commands below.

- Make the simulations each run about a second
- Do your best to optimize the compiler flags and the runtime (openMP) environment

Using the outputs of those runs, estimate the floating point efficiency of our particle-time-step kernel: compare the peak flop/s of the CPU, to the product of particle time steps per second and your estimate of the flops per particle time step. and divide by the throughput of particle time steps per second.  Give that effective arithmetic intensity below.

See above for the plot. We can see that the particle problem is compute bound.

In [40]:
# max 5e8
OMP_PROC_BIND=SPREAD
for chunk in 4
do
    for numthreads in 32
    do
        OMP_NUM_THREADS=$numthreads
        echo "===============NUM THREADS: $numthreads  CHUNK: $(bc <<< "10^$chunk")"
        make clean > /dev/null
        make runcloud NPOINT=64 NT=10000000 NCHUNK=$(bc <<< "10^$chunk") COPTFLAGS="-O3" | grep "/cloud]:"
    done
done

===============NUM THREADS: 32  CHUNK: 10000
[./cloud]: 1.260311e+00 elapsed seconds
[./cloud]: 5.078111e+08 particle time steps per second
[./cloud]: 5.078111e+04 particle time step chunks per second


5e8 time_steps per sec(see above) * 28 flops per step(see Q1) = 14e9 flops per sec

cpu peak: 300e9 flops per sec

effiency: roughly 5%

In [53]:
# max: 9e8
OMP_PROC_BIND=SPREAD
for chunk in 2
do

    for numthreads in 6400
    do
        OMP_NUM_THREADS=$numthreads
        echo "===============NUM THREADS: $numthreads  CHUNK: $chunk"
        make clean > /dev/null
        make runcloud NPOINT=6400000 NT=100 NCHUNK=$chunk COPTFLAGS="-O3 -xHost" | grep "/cloud]:"
    done
done

===============NUM THREADS: 6400  CHUNK: 2
[./cloud]: 6.443222e-01 elapsed seconds
[./cloud]: 9.932919e+08 particle time steps per second
[./cloud]: 4.966460e+08 particle time step chunks per second


1e9 time_steps per sec(see above) * 28 flops per step(see Q1) = 28e9 flops per sec

cpu peak: 300e9 flops per sec

effiency: roughly 10% in this case